In [4]:
nltk.download('punkt', download_dir=r'C:\nltk_data')
nltk.download('stopwords', download_dir=r'C:\nltk_data')
nltk.download('wordnet', download_dir=r'C:\nltk_data')
nltk.download('averaged_perceptron_tagger', download_dir=r'C:\nltk_data')

import nltk
nltk.data.path.append(r'C:\nltk_data')


[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [1]:
# ===============================
# NLP Sentiment Analysis Pipeline 
# ===============================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
import os
import joblib

# -------------------------------
# Setup
# -------------------------------
np.random.seed(42)
os.makedirs('outputs', exist_ok=True)

# -------------------------------
# 1. Load Data
# -------------------------------
df = pd.read_csv('Tweets.csv')
print("Dataset loaded. Shape:", df.shape)

# -------------------------------
# 2. Preprocess Text
# -------------------------------
CUSTOM_STOPWORDS = {"the","a","an","is","in","on","at","to","for","and","or","but","this","that","it","i","you","we","he","she","they"}

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF]', '', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in CUSTOM_STOPWORDS]
    return ' '.join(tokens)

# -------------------------------
# 3. Clean Data
# -------------------------------
df = df[['text', 'airline_sentiment']].dropna()
sentiment_map = {'positive':2, 'neutral':1, 'negative':0}
df['airline_sentiment'] = df['airline_sentiment'].map(sentiment_map)
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("Data cleaned & preprocessed. Shape:", df.shape)

# -------------------------------
# 4. EDA 
# -------------------------------
plt.figure(figsize=(8,6))
sns.countplot(x='airline_sentiment', data=df)
plt.xticks([0,1,2], ['Negative','Neutral','Positive'])
plt.title('Sentiment Distribution')
plt.savefig('outputs/sentiment_distribution.png', dpi=300)
plt.close()

# -------------------------------
# 5. Train-Test Split
# -------------------------------
X = df['cleaned_text']
y = df['airline_sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# 6. TF-IDF Feature Extraction
# -------------------------------
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,3), min_df=2, max_df=0.95)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print("TF-IDF features extracted:", X_train_tfidf.shape)

# ===============================
# 7. Train & Evaluate Models
# ===============================

# Logistic Regression
lr_model = LogisticRegression(max_iter=1000, multi_class='multinomial')
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {acc_lr:.4f}")
cm_lr = confusion_matrix(y_test, y_pred_lr)
plt.figure(figsize=(8,6))
sns.heatmap(cm_lr, annot=True, fmt='d', xticklabels=['Neg','Neu','Pos'], yticklabels=['Neg','Neu','Pos'], cmap='Blues')
plt.title('Confusion Matrix (Logistic Regression)')
plt.savefig('outputs/confusion_matrix_logistic_regression.png', dpi=300)
plt.close()

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {acc_rf:.4f}")
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8,6))
sns.heatmap(cm_rf, annot=True, fmt='d', xticklabels=['Neg','Neu','Pos'], yticklabels=['Neg','Neu','Pos'], cmap='Blues')
plt.title('Confusion Matrix (Random Forest)')
plt.savefig('outputs/confusion_matrix_random_forest.png', dpi=300)
plt.close()

# SVM
svm_model = SVC()
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)
acc_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {acc_svm:.4f}")
cm_svm = confusion_matrix(y_test, y_pred_svm)
plt.figure(figsize=(8,6))
sns.heatmap(cm_svm, annot=True, fmt='d', xticklabels=['Neg','Neu','Pos'], yticklabels=['Neg','Neu','Pos'], cmap='Blues')
plt.title('Confusion Matrix (SVM)')
plt.savefig('outputs/confusion_matrix_svm.png', dpi=300)
plt.close()

# -------------------------------
# 8. Save Best Model & TF-IDF
# -------------------------------
best_acc = max(acc_lr, acc_rf, acc_svm)
if best_acc == acc_lr:
    best_model = lr_model
    best_name = "Logistic Regression"
elif best_acc == acc_rf:
    best_model = rf_model
    best_name = "Random Forest"
else:
    best_model = svm_model
    best_name = "SVM"

print(f"\nBest Model: {best_name} | Accuracy: {best_acc:.4f}")
joblib.dump(best_model, f'outputs/best_model_{best_name.lower().replace(" ","_")}.pkl')
joblib.dump(tfidf, 'outputs/tfidf_vectorizer.pkl')
print("TF-IDF vectorizer saved successfully.")


Dataset loaded. Shape: (14640, 15)
Data cleaned & preprocessed. Shape: (14640, 3)
TF-IDF features extracted: (11712, 5000)


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 0.8002
Random Forest Accuracy: 0.7739
SVM Accuracy: 0.7982

Best Model: Logistic Regression | Accuracy: 0.8002
TF-IDF vectorizer saved successfully.


The dataset contained 14,640 entries and 15 columns, but reducing it to 3 key features retained all essential sentiment information while improving processing efficiency.

Cleaning steps like removing URLs, mentions, hashtags, punctuation, and custom stopwords significantly enhanced text quality and model performance.

TF-IDF vectorization with 5,000 features effectively represented contextual relationships in the tweets, balancing detail and computational cost.

Logistic Regression achieved the highest accuracy of 80.02%, showing that linear models are particularly effective for sparse, high-dimensional TF-IDF text data.

SVM closely followed with 79.82% accuracy, while Random Forest lagged at 77.39%, indicating tree-based methods may be less suited for this type of data.

The saved TF-IDF vectorizer and best-performing model ensure reproducibility and smooth deployment for real-world sentiment analysis tasks.

The overall pipeline—from cleaning and feature extraction to model comparison and artifact saving—demonstrates a robust, production-ready NLP workflow.